In [ ]:
#@markdown # このノートブックについて

#@markdown このノートブックは[litagin02/Style-Bert-VITS2リポジトリにあるGoogle Colabノートブック](https://colab.research.google.com/github/litagin02/Style-Bert-VITS2/blob/master/colab.ipynb)をベースに、Google Driveに接続する機能を除き、声の学習を行なうためだけに簡略化させたノートブックとなります。

#@markdown プログラムが分かる方・より使いこなしていきたい方はベースとさせてもらった [litagin02/Style-Bert-VITS2](https://github.com/litagin02/Style-Bert-VITS2) をご覧ください。


In [ ]:
#@markdown ## Style-Bert-VITS2のセットアップ
#@markdown この処理を実施したら

#@markdown /content/dataset/Style-Bert-VITS2/inputs

#@markdown に、学習させたい音声ファイルを入れてください

!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!pip install -r requirements-colab.txt --timeout 120
!python initialize.py --skip_default_models

# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = "/content/dataset/Style-Bert-VITS2/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = "/content/dataset/Style-Bert-VITS2/model_assets"

# 元となる音声ファイル（wav形式）を入れるディレクトリ
input_dir = "/content/dataset/Style-Bert-VITS2/inputs"

!mkdir -p {dataset_root}
!mkdir -p {assets_root}
!mkdir -p {input_dir}

In [ ]:
import yaml

with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

#@markdown ---
#@markdown # 作成するモデル関連の内容を入力
#@markdown ※デフォルトのままでも問題ありません
# モデル名（話者名）を入力
model_name = "your_model_name" #@param {type:"string"}

# こういうふうに書き起こして欲しいという例文（句読点の入れ方・笑い方や固有名詞等）
initial_prompt = "こんにちは。元気、ですかー？ふふっ、私は……ちゃんと元気だよ！" #@param {type:"string"}

!python slice.py -i {input_dir} --model_name {model_name}
!python transcribe.py --model_name {model_name} --initial_prompt {initial_prompt} --use_hf_whisper

#@markdown ---
#@markdown ## JP-Extraを有効化する
#@markdown JP-Extra （日本語特化版）を有効化すると、日本語の能力が向上する代わりに英語と中国語は使えなくなります。
use_jp_extra = True #@param {type: "boolean"}

#@markdown ---
#@markdown ## 学習のバッチサイズ。
#@markdown VRAMのはみ出具合に応じて調整してください。
batch_size = 4 #@param {type:"string"}

#@markdown ---
#@markdown ## 学習のエポック数
# 100で多すぎるほどかもしれませんが、もっと多くやると質が上がる可能性もあります
epochs = 100 #@param {type:"string"}

#@markdown ---
#@markdown ## 保存頻度
#@markdown 保存頻度。何ステップごとにモデルを保存するか。分からなければデフォルトのままで。
save_every_steps = 1000 #@param {type:"string"}

#@markdown ---
#@markdown 音声ファイルの音量を正規化するかどうか
normalize = False #@param {type:"string"}

#@markdown ---
#@markdown 音声ファイルの開始・終了にある無音区間を削除するかどうか
trim = False #@param {type:"string"}


#@markdown ---
#@markdown 読みのエラーが出た場合にどうするか。

#@markdown ・"raise"ならテキスト前処理が終わったら中断

#@markdown ・"skip"なら読めない行は学習に使わない

#@markdown ・"use"なら無理やり使う
yomi_error = "skip" #@param {type:"string"}

# 以降は学習に関する処理
from gradio_tabs.train import preprocess_all
from style_bert_vits2.nlp.japanese import pyopenjtalk_worker
import yaml
from gradio_tabs.train import get_path

pyopenjtalk_worker.initialize_worker()

preprocess_all(
    model_name=model_name,
    batch_size=batch_size,
    epochs=epochs,
    save_every_steps=save_every_steps,
    num_processes=2,
    normalize=normalize,
    trim=trim,
    freeze_EN_bert=False,
    freeze_JP_bert=False,
    freeze_ZH_bert=False,
    freeze_style=False,
    freeze_decoder=False,
    use_jp_extra=use_jp_extra,
    val_per_lang=0,
    log_interval=200,
    yomi_error=yomi_error,
)

paths = get_path(model_name)
dataset_path = str(paths.dataset_path)
config_path = str(paths.config_path)

with open("default_config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)
yml_data["model_name"] = model_name
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)

if use_jp_extra:
  # 学習 （日本語特化版を「使う」場合）
  !python train_ms_jp_extra.py --config {config_path} --model {dataset_path} --assets_root {assets_root}
else:
  # 学習 （日本語特化版を「使わない」場合）
  !python train_ms.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

In [ ]:
#@markdown # Web UI起動
#@markdown Web UIを起動して学習させたモデルを試す事ができますが、Colabからだと音声の生成ができないかも？
!python app.py --share

In [ ]:
#@markdown # ONNX変換
#@markdown 学習済みのモデルをONMXに変換します。

#@markdown 変換後は

#@markdown /content/dataset/Style-Bert-VITS2/model_assets/{設定したモデル名}/{設定したモデル名}_e100_s300.onnx

#@markdown にモデルが出力されます
!git fetch -p
!git checkout -b dev origin/dev
!pip install -r requirements-colab.txt
!python convert_onnx.py --model /content/dataset/Style-Bert-VITS2/model_assets/your_model_name